# Multilayer Perceptron Regression Model
Predicts the number of comments an article will get based on the article's topic, tone, length, hour of the day and day of the week

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

In [2]:
#Load training data
train_df = pd.read_csv('../data/regression_train_data.csv')

# Extract the columns you want to use as input features
columns = ['BERT_sentiment_score', 'normalised_word_count', 'pub_day', 'pub_hour', 'topic']
# Extract input features from the dataframe
x_train = np.array(train_df[columns])

y_train = np.array(train_df['n_comments'])

In [3]:
# Training and validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [4]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', 
                               # Number of epochs to wait for improvement
                               patience=10,  
                               verbose=1, 
                               # Restore the weights of the best epoch
                               restore_best_weights=True) 

In [5]:
# Hyperparameters
num_features = 5
dropout = 0.2

In [6]:
# Define MLP model architecture
model = Sequential()
# Input layer
model.add(Dense(128, activation='relu', input_dim=num_features))
model.add(BatchNormalization())
model.add(Dropout(dropout))
# Hidden layer
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
# Hidden layer
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
# Output layer
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               768       
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [7]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 377429.3750 - mean_squared_error: 377429.3750 - mean_absolute_error: 310.4794 - val_loss: 325531.4062 - val_mean_squared_error: 325531.4062 - val_mean_absolute_error: 285.9825
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 368005.0625 - mean_squared_error: 368005.0625 - mean_absolute_error: 306.0757 - val_loss: 313683.6875 - val_mean_squared_error: 313683.6875 - val_mean_absolute_error: 281.5088
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 353511.3125 - mean_squared_error: 353511.3125 - mean_absolute_error: 299.0978 - val_loss: 310218.9688 - val_mean_squared_error: 310218.9688 - val_mean_absolute_error: 281.8509
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 335265.3125 - mean_squared_error: 335265.3125 - mean_absolute_error: 291.0441 - val_loss: 280224.7500 - val_mean_squared_error: 280224.7500 - val_mean_absolute_error: 265.0

250/250 [==============================] - 1s 3ms/step - loss: 232105.9375 - mean_squared_error: 232105.9375 - mean_absolute_error: 293.4659 - val_loss: 202940.4844 - val_mean_squared_error: 202940.4844 - val_mean_absolute_error: 271.1512
Epoch 34/100
250/250 [==============================] - 1s 3ms/step - loss: 236070.0781 - mean_squared_error: 236070.0781 - mean_absolute_error: 296.0558 - val_loss: 204467.0000 - val_mean_squared_error: 204467.0000 - val_mean_absolute_error: 269.9231
Epoch 35/100
250/250 [==============================] - 1s 3ms/step - loss: 235452.6562 - mean_squared_error: 235452.6562 - mean_absolute_error: 295.5951 - val_loss: 202229.4844 - val_mean_squared_error: 202229.4844 - val_mean_absolute_error: 263.1682
Epoch 36/100
250/250 [==============================] - 1s 4ms/step - loss: 232615.1094 - mean_squared_error: 232615.1094 - mean_absolute_error: 293.9267 - val_loss: 201140.4688 - val_mean_squared_error: 201140.4688 - val_mean_absolute_error: 274.0835
Epoch

250/250 [==============================] - 1s 4ms/step - loss: 230310.2812 - mean_squared_error: 230310.2812 - mean_absolute_error: 289.5506 - val_loss: 210889.1406 - val_mean_squared_error: 210889.1406 - val_mean_absolute_error: 250.4619
Epoch 66/100
250/250 [==============================] - 1s 5ms/step - loss: 228246.4375 - mean_squared_error: 228246.4375 - mean_absolute_error: 289.2497 - val_loss: 199589.5000 - val_mean_squared_error: 199589.5000 - val_mean_absolute_error: 268.6662
Epoch 67/100
250/250 [==============================] - 1s 4ms/step - loss: 232113.1562 - mean_squared_error: 232113.1562 - mean_absolute_error: 291.3314 - val_loss: 200651.0781 - val_mean_squared_error: 200651.0781 - val_mean_absolute_error: 272.7868
Epoch 68/100
250/250 [==============================] - 1s 4ms/step - loss: 228626.6719 - mean_squared_error: 228626.6719 - mean_absolute_error: 288.5007 - val_loss: 198117.9844 - val_mean_squared_error: 198117.9844 - val_mean_absolute_error: 261.2218
Epoch

In [8]:
# save the trained model
model.save('../models/MLP_regression_model.h5')

In [9]:
# Evaluate the model on validation data
loss, mse, mae = model.evaluate(x_val, y_val)
print('Validation MSE:', mse)
print('Validation MAE:', mae)

63/63 [==============================] - 0s 1ms/step - loss: 194532.0000 - mean_squared_error: 194532.0000 - mean_absolute_error: 270.4090
Validation MSE: 194532.0
Validation MAE: 270.40899658203125
